In [1]:
from datasets import load_from_disk

data_path2 = "/opt/ml/input/data/test_dataset"
dataset = load_from_disk(data_path2)
dataset

DatasetDict({
    validation: Dataset({
        features: ['id', 'question'],
        num_rows: 600
    })
})

In [11]:
dataset["validation"].data.to_pandas()

,question,id
0,우르바노 5세는 1370년에 어디로 가기 위해 코르네토에서 배를 탔나요?,mrc-1-000001
1,유엔 인구 기금은 누구를 만날 때 7개의 규칙을 따르라고 하였나?,mrc-1-000002
2,신탁통치 오보사건은 어떤 신문사와 관련되어있나요?,mrc-1-000003
3,1972년에 실시된 서독 총선거에서 뽑아야 하는 서베를린의 의원 수는?,mrc-1-000005
4,웰링턴 부대는 살라망카 전투에서 승리한 후 어디로 진출하였나요?,mrc-1-000006
...,...,...
9579,뉴크로부존의 마법 사용자들의 이명은?,mrc-0-005506
9580,여운형에 대한 생각이 적대적이였다가 긍정적으로 바뀐 조직은?,mrc-0-005507
9581,선발 투수로서 첫 해에 젱킨스의 삼진 갯수는?,mrc-0-005509
9582,재일여대생이 여운형 이후에 만난 남자의 정체는?,mrc-0-005512


In [2]:
import os
import torch
from korbert.src_tokenizer.tokenization_morp import whitespace_tokenize, BasicTokenizer, BertTokenizer
from pytorch_pretrained_bert.modeling import BertForQuestionAnswering, BertConfig


model_name = "bert-large-cased"
model_path = "/opt/ml/code/korbert/pytorch_model.bin"
config_path = "/opt/ml/code/korbert/bert_config.json"

config = BertConfig(config_path)
model = BertForQuestionAnswering(config)
state_dict = torch.load(os.path.join(model_name, model_path))

del_list = ["cls.predictions.bias", "cls.predictions.transform.dense.weight", "cls.predictions.transform.dense.bias", "cls.predictions.transform.LayerNorm.weight", "cls.predictions.transform.LayerNorm.bias", "cls.predictions.decoder.weight", "cls.seq_relationship.weight", "cls.seq_relationship.bias"]
for elt in del_list:
    state_dict.pop(elt)
state_dict["qa_outputs.weight"] = torch.zeros([2, 768])
state_dict["qa_outputs.bias"] = torch.zeros([2])

model.load_state_dict(state_dict)

<All keys matched successfully>

In [6]:
from korbert.src_tokenizer.tokenization_morp import whitespace_tokenize, BasicTokenizer, BertTokenizer
#tokenizer = BasicTokenizer()

vocab_path = "/opt/ml/code/korbert/vocab.korean_morp.list"
tokenizer_path = os.path.join(model_name, vocab_path)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

In [35]:
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE

state_dict = torch.load( os.path.join(model_name, model_path) )
model = BertForQuestionAnswering.from_pretrained(
    model_name,
    state_dict=state_dict,
    cache_dir=PYTORCH_PRETRAINED_BERT_CACHE / 'distributed_{}'.format(-1)
)

100%|██████████| 1242874899/1242874899 [01:48<00:00, 11453539.41B/s]


RuntimeError: Error(s) in loading state_dict for BertForQuestionAnswering:
	size mismatch for bert.embeddings.word_embeddings.weight: copying a param with shape torch.Size([30349, 768]) from checkpoint, the shape in current model is torch.Size([28996, 1024]).
	size mismatch for bert.embeddings.position_embeddings.weight: copying a param with shape torch.Size([512, 768]) from checkpoint, the shape in current model is torch.Size([512, 1024]).
	size mismatch for bert.embeddings.token_type_embeddings.weight: copying a param with shape torch.Size([2, 768]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
	size mismatch for bert.embeddings.LayerNorm.weight: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for bert.embeddings.LayerNorm.bias: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for bert.encoder.layer.0.atte